In [2]:
import os
import sys

current_dir = os.path.abspath('') 
root_dir = os.path.join(current_dir, "../")
sys.path.insert(0, root_dir)

print(sys.path)
import pandas as pd
import numpy as np
from sklearn import model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import string

import utils, config 
from engine import engine_lr, engine_nn
from processing import processing, emb_processing

import tensorflow.keras as keras
import tensorflow as tf
from keras.layers import LSTM, Dense, Flatten


from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
import re
from sklearn.preprocessing import LabelEncoder


%load_ext autoreload
%autoreload 2

['/home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../', '', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python36.zip', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/lib-dynload', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/IPython/extensions', '/home/cedric/.ipython']


Using TensorFlow backend.


In [10]:
sadness_data = pd.read_csv("../inputs/edited_data/new_sadness.csv")
joy_data = pd.read_csv("../inputs/edited_data/new_joy.csv")
anger_data = pd.read_csv("../inputs/edited_data/new_anger.csv")
love_data = pd.read_csv("../inputs/edited_data/new_love.csv")

data = pd.concat([sadness_data, joy_data, love_data, anger_data])

In [11]:
data["sentiment"] = data["sentiment"].apply(lambda x: str(x).strip())
data["sentiment"] = data["sentiment"].apply(lambda x: "anger" if x=="hate" else x)

data = data[data.sentiment.isin(["sadness", "anger", "joy", "love"])] 

data["text"] = data["text"].apply(lambda x: utils.clean_tweets(x))

data_merge = data.copy()

In [12]:

# Devide data set between train and test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data_merge.drop(config.TARGET_COL, axis=1),
    data_merge[config.TARGET_COL],
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=data_merge[config.TARGET_COL].values
)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

y_val_encoded = label_encoder.transform(y_test)


In [13]:
data["sentiment"].value_counts()

sadness    643
anger      569
joy        415
love       349
Name: sentiment, dtype: int64

In [14]:
label_encoder.inverse_transform([0,1,2,3])

array(['anger', 'joy', 'love', 'sadness'], dtype=object)

In [15]:
class_weight = {
    0: 643/569,
    1: 643/415,
    2: 643/349,
    3: 643/643
}

# NN + embedding models

In [16]:
data_merge = data_merge.sample(frac=1)

#x_clean = preprocess_func.transform(X)
x_clean = emb_processing.clean_text(X_train)
x_input, tokenizer = emb_processing.tokenize_text(x_clean, istraining=True)

x_test_clean = emb_processing.clean_text(X_test)
x_test_input, _ = emb_processing.tokenize_text(x_test_clean, tokenizer=tokenizer)


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_labels = keras.utils.to_categorical(y_encoded)

y_test_encoded = label_encoder.transform(y_test)
y_test_labels = keras.utils.to_categorical(y_test_encoded)

In [17]:
x_clean.head(30)

328     i love it when i feel hot and beautiful and sexy
143    i know it be the holy spirit at work plus it f...
290                  i be feel frustrated and tire today
537    i be not amazing or great at photography but i...
433            i be feel a combination of smug and happy
513    i can not help but feel proud and grateful to ...
250       i feel so honor to have be a part of this year
131    oh thunderstorm boo why do they always have to...
51     id just have a terrible nightmare and be feel ...
561    i come to this realization that i be often fee...
381    where be the good music and lyric it seem like...
426    i want to scream to yell at everyone who i fee...
197    i feel horrible because i feel horrible make w...
306    i wish she know what she put me through she st...
560    i be feel melancholy i will embrace it and lis...
214    i feel thrill to be able to investigate my own...
404                                          i suck dick
580       people fall in love v

In [18]:
_, _, word_emb = emb_processing.read_embedded_vecs(config.EMBEDDING_FILE_PATH)

In [19]:
limite_apparition = 2
from collections import Counter
results = Counter()
x_clean.str.split().apply(results.update)
results = results.most_common()
reccurent_words = [tuple[0] for tuple in results if (tuple[1]>= limite_apparition) and (tuple[0] not in word_emb.keys())]
#print([tuple for tuple in results if tuple[0] not in word_emb.keys()])

In [20]:
reccurent_words

['siwons', 'anyones', 'bummin']

In [21]:
emb_matrix = emb_processing.get_embedding_matrix(word_emb, tokenizer, reccurent_words=reccurent_words)

In [38]:
import tensorflow as tf
from tensorflow_addons.metrics import MatthewsCorrelationCoefficient
np.random.seed(1)
tf.random.set_seed(2)

n_classes = 4
epochs = 15
model_1 = engine_nn.FeedForwardNn(embedding_matrix=emb_matrix, n_classes=n_classes)
model_2 = engine_nn.LstmNetwork(embedding_matrix=emb_matrix, n_classes=n_classes)
model_3 = engine_nn.MixNetwork(embedding_matrix=emb_matrix, n_classes=n_classes)

opt = keras.optimizers.Adam(learning_rate=0.003)
model_1.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=[utils.mcc_metric, "acc"])
model_2.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=[utils.mcc_metric, "acc"])
model_3.compile(loss='categorical_crossentropy',optimizer=opt, metrics=[utils.mcc_metric, "acc"])

#model_1.fit(x=x_input, y=y_labels, verbose=2, validation_data=(x_test_input, y_test_labels), epochs=epochs, class_weight=class_weight)
#model_2.fit(x=x_input, y=y_labels, verbose=2, validation_data=(x_test_input, y_test_labels), epochs=epochs, class_weight=class_weight)
model_3.fit(x=x_input, y=y_labels, verbose=2, validation_data=(x_test_input, y_test_labels), epochs=epochs, class_weight=class_weight)

Epoch 1/15
50/50 - 2s - loss: 1.7606 - mcc_metric: 0.0533 - acc: 0.2873 - val_loss: 1.3067 - val_mcc_metric: 0.1806 - val_acc: 0.3838
Epoch 2/15
50/50 - 1s - loss: 1.5667 - mcc_metric: 0.2072 - acc: 0.4032 - val_loss: 1.1222 - val_mcc_metric: 0.2350 - val_acc: 0.4268
Epoch 3/15
50/50 - 1s - loss: 1.4584 - mcc_metric: 0.2600 - acc: 0.4443 - val_loss: 1.0861 - val_mcc_metric: 0.2489 - val_acc: 0.4419
Epoch 4/15
50/50 - 1s - loss: 1.3593 - mcc_metric: 0.3239 - acc: 0.4949 - val_loss: 1.0662 - val_mcc_metric: 0.2831 - val_acc: 0.4646
Epoch 5/15
50/50 - 1s - loss: 1.1983 - mcc_metric: 0.4328 - acc: 0.5734 - val_loss: 0.9470 - val_mcc_metric: 0.4786 - val_acc: 0.6061
Epoch 6/15
50/50 - 1s - loss: 1.0988 - mcc_metric: 0.5131 - acc: 0.6354 - val_loss: 0.8575 - val_mcc_metric: 0.5449 - val_acc: 0.6540
Epoch 7/15
50/50 - 1s - loss: 0.9756 - mcc_metric: 0.6175 - acc: 0.7127 - val_loss: 0.8371 - val_mcc_metric: 0.5438 - val_acc: 0.6616
Epoch 8/15
50/50 - 2s - loss: 0.8378 - mcc_metric: 0.6606 - ac

In [25]:
print(model_1.evaluate(x_test_input, y_test_labels))
print(model_2.evaluate(x_test_input, y_test_labels))
print(model_3.evaluate(x_test_input, y_test_labels))

13/13 [==============================] - 0s 2ms/step - loss: 1.4859 - mcc_metric: 0.4679 - acc: 0.5934
[1.4858554601669312, 0.4679487347602844, 0.5934343338012695]
13/13 [==============================] - 0s 12ms/step - loss: 1.8142 - mcc_metric: 0.4423 - acc: 0.5859
[1.8141738176345825, 0.4423076808452606, 0.5858585834503174]
13/13 [==============================] - 0s 7ms/step - loss: 0.9414 - mcc_metric: 0.5705 - acc: 0.6742
[0.941415548324585, 0.5705128312110901, 0.6742424368858337]


# Inspect results on train

In [ ]:
predictions = model.predict(x_input)

winning_class = np.apply_along_axis(lambda x: np.argmax(x), 1, predictions)
winning_class_pred = np.apply_along_axis(lambda x: np.max(x), 1, predictions)
winning_class_name = label_encoder.inverse_transform(winning_class)

train_copie = X_train.copy()
train_copie["clean"] = x_clean.values
train_copie["value"] = y_train.values
train_copie["pred"] = winning_class_name
train_copie["pred_prop"] = winning_class_pred

train_copie["match"] = train_copie.apply(lambda x: int(x.value==x.pred), axis=1)
train_copie.head()

In [ ]:
print(train_copie[train_copie.match==1].value.value_counts())
print(train_copie[train_copie.match==0].value.value_counts())
pd.crosstab(train_copie.value, train_copie.pred)

# Inspect results on raw exmple

In [51]:
sentence_test = "like "

test_clean = emb_processing.clean_text(pd.DataFrame({config.TEXT_COL:[sentence_test]}))
print(test_clean)
x_test, _ = emb_processing.tokenize_text(test_clean, istraining=False, tokenizer=tokenizer)

print(model_3.predict(x_test))

print(label_encoder.inverse_transform([0, 1, 2, 3]))

0    like
Name: text, dtype: object
[[0.22524747 0.28871542 0.17468892 0.31134823]]
['anger' 'joy' 'love' 'sadness']


# Save model

In [57]:
import pickle
from pathlib import Path

# Create the directory for the model
Path(config.CUSTOM_DATA_DIR).mkdir(parents=True, exist_ok=True)


model_to_keep = model_3


label_dict = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))


pickle.dump(label_dict, open(config.CUSTOM_DATA_LABEL_DICT_PATH, "wb"))
pickle.dump(tokenizer, open(config.CUSTOM_DATA_TOKENIZER_PATH, "wb"))
tf.keras.models.save_model(
    model_to_keep, config.CUSTOM_DATA_MODEL_PATH, overwrite=True, include_optimizer=True, save_format="tf",
    signatures=None, options=None
)

INFO:tensorflow:Assets written to: /home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../models/custom_data/custom_data_model/assets
